In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import os
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np 
import glob
import time
import yfinance as yf
import datetime
import requests
import shutil


ERROR = False

cookies_ec_dict = {'google': EC.element_to_be_clickable((By.XPATH, "//div[@class='QS5gu sy4vM']")),
                'nasdaq': EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")),
                'investing.com':EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")),
                'zacks.com_readmore': EC.visibility_of_element_located((By.XPATH, "//button[contains(@class, 'Button__StyledButton') and contains(text(), 'Read more to accept preferences')]")),
                'zacks.com': EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'Button__StyledButton-a1qza5-0') and contains(text(), 'Accept all')]")),
                'tipranks.com': EC.element_to_be_clickable((By.CLASS_NAME, "Button__StyledButton-a1qza5-0"))}

nav_nasdaq_ec_dict = {'select_date': EC.element_to_be_clickable((By.ID, "date-picker__toggle")),
                            #"//button[@class='date-picker__toggle' and @aria-label='click show datepicker popup']")),      
                      'select_searchbox':  EC.visibility_of_element_located((By.CLASS_NAME, "date-picker__input")),
                      'apply_date_button': EC.element_to_be_clickable((By.XPATH,
                            "//button[@class='date-picker__apply']"))}

nav_investingcom_ec_dict = {'select_calendar': EC.element_to_be_clickable((By.ID, "datePickerToggleBtn"))}

buysell_tipranks_dict = {'Strong Buy': '1', 'Moderate Buy': '2', 'Hold': '3', }

def accept_cookies(site = 'google'):   
    if site == 'zacks.com':
        try:
            read_more = WebDriverWait(driver, 5).until(cookies_ec_dict['zacks.com_readmore']) 
            read_more.click()
        except:
            print("No 'Read more' button found on ", site)
        try:
            accept = WebDriverWait(driver, 5).until(cookies_ec_dict[site])
            accept.click()
        except:
            print("No 'Accept' button found or unable to click on ", site)
    
    elif site == 'tipranks.com':
        try:
            accept_button = WebDriverWait(driver, 5).until(cookies_ec_dict['tipranks.com'])
            accept_button.click()
            print("Accept button clicked successfully")
        except Exception as e:
            print("Error:", e)

    elif site == 'stockinvest.us':
        pass

    elif site == 'tradingview.com':
        pass

    elif site == 'investing.com':
        accept = WebDriverWait(driver, 5).until(cookies_ec_dict[site])
        accept.click()



def google(query, cookies):
    # Open the website
    driver.get("https://google.com")
    if cookies: accept_cookies('google')
    search_box = driver.find_element(By.NAME, "q")

     # Enter your search query
    search_query = query
    search_box.send_keys(search_query)

    # Simulate hitting Enter
    search_box.send_keys(Keys.RETURN)

def click_google_site(site):
    driver.find_element(By.XPATH, f"//a[contains(@href, '{site}')]").click()

def select_date(year, month, day, site = 'nasdaq'):
    if site == 'nasdaq':
        date = datetime.date(year, month, day).strftime('%m/%d/%Y')
        WebDriverWait(driver, 1).until(nav_nasdaq_ec_dict['select_date']).click()
        driver.execute_script(f"document.querySelector('.date-picker__input').value = '{date}';")
        WebDriverWait(driver, 1).until(nav_nasdaq_ec_dict['e_button']).click()
    elif site == 'investing.com':
        date = datetime.date(year, month, day).strftime('%m/%d/%Y')
        WebDriverWait(driver, 1).until(nav_investingcom_ec_dict['select_calendar']).click()
        input_element = driver.find_element(By.CSS_SELECTOR, "input.newInput#startDate")
        input_element.clear()  # Clear any existing value
        input_element.send_keys(date)  
        input_element = driver.find_element(By.CSS_SELECTOR, "input.newInput#endDate")
        input_element.clear()  # Clear any existing value
        input_element.send_keys(date)  
        driver.find_element(By.ID, 'applyBtn').click()
            
def google_site(query, site, cookies):
    google(query, cookies)
    click_google_site(site)


def get_symbols(site, openclose = False):
    time.sleep(1)

    symbols = []
    if openclose: timings = []
    if site == 'investing.com':
        openclose_dict = {'1': 'open', '2': 'none', '3': 'close'}
        table = driver.find_element(By.ID, "earningsCalendarData")
        tbody = table.find_element(By.TAG_NAME, "tbody")
        rows = tbody.find_elements(By.TAG_NAME, "tr")
        # Iterate over each table row
        for index, row in enumerate(rows):
            if index == 0: continue
            try:
                inner_html = row.get_attribute("innerHTML")
                soup = BeautifulSoup(inner_html, "html.parser")
                a_element = soup.find("a", class_="bold middle")
                if openclose:
                    timing_element = soup.find("td", class_="right time")
                    timing_value = timing_element.get("data-value")
                    timings.append(openclose_dict[timing_value])
                if a_element:
                    company = a_element.text.strip()
                    symbols.append(company)
            except Exception as e:
                print(f"Error processing row {index}: {e}")
                continue
    if openclose: return symbols, timings
    return symbols

def search_on(website, query):
    if website == 'zacks.com':
        search_input = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.ID, "search-q"))
        )
        # Input the search query
        search_input.send_keys(query)
        # Press ENTER to submit the form
        search_input.send_keys(Keys.ENTER)


def get_rank(website):
    time.sleep(0.5)

    if website == 'zacks.com':
        try:
            # Wait for the paragraph element to be visible
            rank_paragraph = WebDriverWait(driver, 2).until(
                EC.visibility_of_element_located((By.CLASS_NAME, "rank_view"))
            )

            # Get the text of the paragraph element
            rank_text = rank_paragraph.text.strip()
            return int(rank_text[0])
        
        except Exception as e:
            ERROR = True


    elif website == 'tradingview.com':
        try:
            speedometer_div = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, ".speedometerWrapper-kg4MJrFB.summary-kg4MJrFB"))
            )
            # Get the outer HTML content of the div element
            html_content = speedometer_div.get_attribute("outerHTML")
            soup = BeautifulSoup(html_content, 'html.parser')
            counter_spans = soup.find_all('span', class_='counterNumber-kg4MJrFB')
            numbers = [int(span.text) for span in counter_spans]
            if numbers is not None and sum(numbers)!=0:
                return (-numbers[0]+numbers[-1])/sum(numbers)
        except Exception as e:
            print('falha')
            ERROR = True
            

    elif website == 'tipranks.com':
        try:
            rank_span = WebDriverWait(driver, 2).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, ".colorpale.fonth4_bold.aligncenter"))
            )

            # Get the text of the span element
            rank_text = rank_span.text.strip()
            return rank_text
        except Exception as e:
            ERROR = True

    elif website == 'stockinvest.us':
        try:
            technicals_rank = WebDriverWait(driver, 5).until(
                EC.visibility_of_element_located((By.XPATH, "//span[@class='pl-5 pr-5 pt-3 pb-3 font-weight-600 border-radius-default font-size-16']"))
            )

            # Get the text of the element
            technicals_rank = number_element.text.strip()
            print(technicals_rank)
            return technicals_rank      
        except Exception as e:
            print('falha')
            ERROR = True




            
def analyst(website, symbol, cookies):
    if website == 'zacks.com':
        driver.get(f'https://www.zacks.com/stock/quote/{symbol}?q={symbol}')
    elif website == 'tipranks.com':
        driver.get(f'https://www.tipranks.com/stocks/{symbol}/forecast')
    elif website == 'stockinvest.us':
        driver.get(f'https://stockinvest.us/stock/{symbol}')
    elif website == 'tradingview.com':
        driver.get(f'https://www.tradingview.com/symbols/{symbol}/technicals')
    elif website == 'stockanalysis.com':
        driver.get(f'https://www.stockanalysis.com/stocks/{symbol}/forecast/')
    else:
        google_site(f'{website[:website.find(".")]} {symbol}', website, cookies)
    if cookies: accept_cookies(website)
    rank = get_rank(website)
    return rank
   
def earnings_rank(website, year, month, day, path = ''):
    if path != '':
        if path[-1] == '/': path = path[:-1]
    if os.path.exists(path+'/'+f'{year}-{month}-{day}') is False:   
        print('path does not exist')
    else:
        for index, tickerpath in enumerate(glob.glob(path+'/'+f'{year}-{month}-{day}/*/')):
            ticker = tickerpath[len(path+'/'+f'{year}-{month}-{day}/'):-1]
            rank = analyst(website, ticker, cookies = not(index))
            print(ticker, rank)
            with open(tickerpath+website, 'a') as f:
                f.write(f'{datetime.datetime.today().year}-{datetime.datetime.today().month}-{datetime.datetime.today().day},    {rank}\n')


def get_tickers_for_earnings_date(year, month, day, openclose = False):
    #google_site('earnings calendar', 'investing.com', cookies = True)
    driver.get('https://www.investing.com/earnings-calendar/')
    time.sleep(0.5)
    accept_cookies('investing.com')
    time.sleep(0.5)
    select_date(year, month, day, site = 'investing.com')
    if openclose: 
        tickers, timings = get_symbols('investing.com', openclose = openclose)
        return tickers, timings
    else:
        tickers = get_symbols('investing.com', openclose = openclose)
        return tickers
 

def make_earnings_dates_directories(path, year, month, day, tickers, openclose  = None):
    if path[-1] == '/': path = path[:-1]
    if os.path.exists(path) is False:
        os.mkdir(path)
    if os.path.exists(path+'/'+f'{year}-{month}-{day}') is False:
        os.mkdir(path+'/'+f'{year}-{month}-{day}')  
    for index, ticker in enumerate(tickers):
        if os.path.exists(path+'/'+f'{year}-{month}-{day}/'+ ticker) is False:
            os.mkdir(path+'/'+f'{year}-{month}-{day}/'+ ticker)
        if openclose is not None:
            with open(path+'/'+f'{year}-{month}-{day}/'+ ticker + '/openclose.dat', 'a') as f:
                f.write(f'{openclose[index]} \n')

    
         
def filter_by_lastrank(path, year, month, day, website):
    if path != '':
        if path[-1] == '/': path = path[:-1]
    if os.path.exists(path+'/'+f'{year}-{month}-{day}') is False:   
        print('path does not exist')
    else:
        ranks = ['1','2','3','4','5','None']
        ticker_rank = {rank:[] for rank in ranks}
        if website == 'zacks.com':
            for _, tickerpath in enumerate(glob.glob(path+'/'+f'{year}-{month}-{day}/*/')):
                ticker = tickerpath[len(path+'/'+f'{year}-{month}-{day}/'):-1]
                if os.path.exists(tickerpath+website):
                    with open(tickerpath+website, 'r') as f:
                        text = f.read()
                        last = text.split(' ')[-1]
                        ticker_rank[last[:last.find('\n')]].append(ticker)


        elif website == 'tradingview.com':
            for _, tickerpath in enumerate(glob.glob(path+'/'+f'{year}-{month}-{day}/*/')):
                ticker = tickerpath[len(path+'/'+f'{year}-{month}-{day}/'):-1]
                if os.path.exists(tickerpath+website):
                    with open(tickerpath+website, 'r') as f:
                        text = f.read()
                        try:
                            last = float(text.split(' ')[-1])
                            if last > 0.5:
                                ticker_rank['1'].append(ticker)
                            elif last <= 0.5 and last > 0.25:
                                ticker_rank['2'].append(ticker)
                            elif last >= -0.25 and last <= 0.25:
                                ticker_rank['3'].append(ticker)
                            elif last >= -0.5 and last < -0.25:
                                ticker_rank['4'].append(ticker)
                            elif last < -0.5:
                                ticker_rank['5'].append(ticker)
                        except:
                            ticker_rank['None'].append(ticker)


                    
        return ticker_rank

     

def get_tickers_on_trading212():
    url = "https://live.trading212.com/api/v0/equity/metadata/instruments"

    headers = {"Authorization": f"{open('.apikey2', 'r').read()[:-1]}"}

    response = requests.get(url, headers=headers)

    data = response.json()

    trading212_tickers = []
    for instrument in data:
        trading212_tickers.append(instrument['shortName']) 

    return trading212_tickers

def update_reporting_tickers(year, month, day, new_tickers):
    
    directory = f'earnings/{year}-{month}-{day}/'
    companies_already_saved = []
    for company in glob.glob(directory + '*/'):
        tic = company[company.find(directory)+len(directory):]
        tic = tic[:tic.find('/')]
        if tic not in new_tickers:
            shutil.rmtree(company)
##########################################


In [2]:
#trading212_tickers = get_tickers_on_trading212()
trading212_tickers = list(np.load('trading212tickers.npy'))

In [11]:
driver = webdriver.Chrome()
year, month, day = 2024, 6, 4
tickers, timings = get_tickers_for_earnings_date(year, month, day, openclose = True)
driver.quit()

In [12]:
for tic, tim in zip(tickers, timings):
    print(tic, tim)
print(len(tickers))

CRWD close
FERG open
HPE close
BBWI open
CNM open
GWRE close
DCI open
PVH close
VRNT close
DBI open
LILM none
CVGW none
VLGEA none
SFIX close
CTRN open
SPWH close
LAKE close
TOUR open
OCCI none
PNF none
QMCO none
LINUF none
22


In [13]:
   
directory = f'earnings/{year}-{month}-{day}/'
companies_already_saved = []
print('Saved but not reporting tomorrow')
for company in glob.glob(directory + '*/'):
    tic = company[company.find(directory)+len(directory):]
    tic = tic[:tic.find('/')]
    companies_already_saved.append(tic)
    if tic not in tickers:
        print(tic)

print('\n')
print('Not saved, but reporting tomorrow')
for tic in tickers:
    if tic not in companies_already_saved:
        print(tic)
        

Saved but not reporting tomorrow


Not saved, but reporting tomorrow
CRWD
FERG
HPE
BBWI
CNM
GWRE
DCI
PVH
VRNT
DBI
LILM
CVGW
VLGEA
SFIX
CTRN
SPWH
LAKE
TOUR
OCCI
PNF
QMCO
LINUF


In [14]:
update_reporting_tickers(year, month, day, tickers)

In [15]:
make_earnings_dates_directories('earnings', year, month, day, tickers, openclose = timings)


In [16]:
#year, month, day = 2024, 4, 1
driver = webdriver.Chrome()
#earnings_rank('zacks.com', year, month, day, path = 'earnings')
earnings_rank('tradingview.com', year, month, day, path = 'earnings')
driver.quit()

GWRE -0.2692307692307692
CNM -0.07692307692307693
LINUF -0.5384615384615384
QMCO -0.5769230769230769
CRWD -0.19230769230769232
BBWI 0.6153846153846154
DBI 0.5769230769230769
PVH 0.5384615384615384
PNF -0.2692307692307692
HPE 0.08
LILM -0.4230769230769231
VLGEA 0.38461538461538464
OCCI 0.2692307692307692
FERG -0.15384615384615385
LAKE 0.4230769230769231
CVGW -0.3076923076923077
VRNT -0.3076923076923077
DCI 0.11538461538461539
TOUR 0.5384615384615384
SPWH 0.46153846153846156
SFIX 0.34615384615384615
CTRN 0.3076923076923077


In [17]:
driver = webdriver.Chrome()
earnings_rank('zacks.com', year, month, day, path = 'earnings')
driver.quit()

GWRE 3
CNM 3
LINUF None
QMCO 3
CRWD 2
BBWI 2
DBI 3
PVH 3
PNF None
HPE 3
LILM 3
VLGEA None
OCCI 3
FERG 3
LAKE 1
CVGW 3
VRNT 4
DCI 2
TOUR None
SPWH 2
SFIX 3
CTRN None


In [30]:
#year, month, day = 2024, 4, 1
ticker_rank_tradingview = filter_by_lastrank('earnings', year, month, day, 'tradingview.com')
ticker_rank_zacks = filter_by_lastrank('earnings', year, month, day, 'zacks.com')

up_to_ranking = 3

In [31]:
ticker_rank_tradingview

{'1': ['BBWI', 'DBI', 'PVH', 'TOUR'],
 '2': ['VLGEA', 'OCCI', 'LAKE', 'SPWH', 'SFIX', 'CTRN'],
 '3': ['CNM', 'CRWD', 'HPE', 'FERG', 'DCI'],
 '4': ['GWRE', 'PNF', 'LILM', 'CVGW', 'VRNT'],
 '5': ['LINUF', 'QMCO'],
 'None': []}

In [32]:
## zacks
total_number_of_companies = 0
zacks_companies = []
for i in range(1,up_to_ranking + 1):
    ticker_rank_on212 = [x for x in ticker_rank_zacks[f'{i}'] if x in trading212_tickers]
    zacks_companies.extend(ticker_rank_on212)
    print(f'{i}: ', ticker_rank_on212, '\n')
    total_number_of_companies += len(ticker_rank_on212)

with open(f'earnings/{year}-{month}-{day}/_zacks_rank{up_to_ranking}.txt', 'w+') as f:
    for company in zacks_companies:
        f.write(f'{company} \n')


print('total number of companies: ', total_number_of_companies)

1:  ['LAKE'] 

2:  ['CRWD', 'BBWI', 'DCI', 'SPWH'] 

3:  ['GWRE', 'QMCO', 'DBI', 'PVH', 'HPE', 'LILM', 'FERG', 'CVGW', 'SFIX'] 

total number of companies:  14


In [33]:
## tradingview
total_number_of_companies = 0
tradingview_companies = []
for i in range(1,up_to_ranking + 1):
    ticker_rank_on212 = [x for x in ticker_rank_tradingview[f'{i}'] if x in trading212_tickers]
    tradingview_companies.extend(ticker_rank_on212)
    print(f'{i}: ', ticker_rank_on212, '\n')
    total_number_of_companies += len(ticker_rank_on212)

with open(f'earnings/{year}-{month}-{day}/_tradingview_rank{up_to_ranking}.txt', 'w+') as f:
    for company in tradingview_companies:
        f.write(f'{company} \n')
    f.write(f'{len(tradingview_companies)}')


print('total number of companies: ', total_number_of_companies)

1:  ['BBWI', 'DBI', 'PVH'] 

2:  ['VLGEA', 'LAKE', 'SPWH', 'SFIX', 'CTRN'] 

3:  ['CRWD', 'HPE', 'FERG', 'DCI'] 

total number of companies:  12


In [34]:
intersection_companies = list(set(zacks_companies) & set(tradingview_companies))
with open(f'earnings/{year}-{month}-{day}/_intersection_rank{up_to_ranking}.txt', 'w+') as f:
    for company in intersection_companies:
        f.write(f'{company} \n')
    f.write(f'{len(intersection_companies)}')
intersection_companies

['DCI', 'DBI', 'SPWH', 'CRWD', 'HPE', 'PVH', 'BBWI', 'LAKE', 'SFIX', 'FERG']

In [15]:
invested_initial_value = 1033.85
initial_value_per_ticker = invested_initial_value/len(zacks_companies)
total_initial_value = 0
total_final_value = 0
total_percentage = 0
start_date = str(datetime.date(year, month, day) - datetime.timedelta(days = 10))
if datetime.date(year, month, day).weekday() == 4:
    end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 4))
else:
    end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 2))

for tic in zacks_companies:
    yf_ticker = yf.download(tickers=tic, start=start_date, end =end_date, progress=False, ignore_tz=True)
    print('\n')
    print(tic)
    print(yf_ticker['Open'])
    open_values = yf_ticker['Open'].values
    initial_value = open_values[-3]
    final_value = open_values[-1]
    
    ## take some value when market is closed
    #yf_ticker = yf.download(tickers=tic, period = '1d', interval = '1m', progress=False, ignore_tz=True,  prepost= True)
    #final_value= yf_ticker['Open'].values[-1]
    
    percentage_difference = (final_value - initial_value)/initial_value*100
    print(f'{tic}, {np.round(initial_value, 2)},   {np.round(final_value, 2)},  \
            {np.round(initial_value_per_ticker*percentage_difference,2)},    \
            {np.round(initial_value_per_ticker + (initial_value_per_ticker*percentage_difference/100),2)}')
    total_initial_value += initial_value_per_ticker
    total_final_value += initial_value_per_ticker+(initial_value_per_ticker*percentage_difference/100)

# compare with spy
yf_ticker = yf.download(tickers='SPY', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_spy = (open_values[-1] - open_values[-3])/open_values[-3]*100

# compare with nasdaq
yf_ticker = yf.download(tickers='NDAQ', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_ndaq = (open_values[-1] - open_values[-3])/open_values[-3]*100


print('\n', total_initial_value, np.round(total_final_value,2), \
      np.round((total_final_value - total_initial_value)/total_initial_value*100,2), \
      np.round(percentage_difference_spy,2), np.round(percentage_difference_ndaq,2))

print(end_date)

ZeroDivisionError: float division by zero

In [21]:
invested_initial_value = 1011.28
initial_value_per_ticker = invested_initial_value/len(tradingview_companies)
total_initial_value = 0
total_final_value = 0
total_percentage = 0
if datetime.date(year, month, day).weekday() == 4:
    end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 4))
else:
    end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 2))
for tic in tradingview_companies:
    yf_ticker = yf.download(tickers=tic, start=start_date, end =end_date, progress=False, ignore_tz=True)
    open_values = yf_ticker['Open'].values
    initial_value = open_values[-3]
    final_value = open_values[-1]
    print('\n')
    print(tic)
    print(yf_ticker['Open'])
    
    ## take some value when market is closed
    #yf_ticker = yf.download(tickers=tic, period = '1d', interval = '1m', progress=False, ignore_tz=True,  prepost= True)
    #final_value= yf_ticker['Open'].values[-1]
    
    percentage_difference = (final_value - initial_value)/initial_value*100
    print(f'{tic}, {np.round(initial_value, 2)},   {np.round(final_value, 2)},  \
            {np.round(initial_value_per_ticker*percentage_difference,2)},    \
            {np.round(initial_value_per_ticker + (initial_value_per_ticker*percentage_difference/100),2)}')
    total_initial_value += initial_value_per_ticker
    total_final_value += initial_value_per_ticker+(initial_value_per_ticker*percentage_difference/100)

# compare with spy
yf_ticker = yf.download(tickers='SPY', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_spy = (open_values[-1] - open_values[-3])/open_values[-3]*100

# compare with nasdaq
yf_ticker = yf.download(tickers='NDAQ', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_ndaq = (open_values[-1] - open_values[-3])/open_values[-3]*100


print('\n', total_initial_value, np.round(total_final_value,2), \
      np.round((total_final_value - total_initial_value)/total_initial_value*100,2), \
      np.round(percentage_difference_spy,2), np.round(percentage_difference_ndaq,2))

print(end_date)



MESA
Date
2024-05-13    0.92
2024-05-14    0.87
2024-05-15    0.89
2024-05-16    0.88
2024-05-17    0.89
2024-05-20    1.01
2024-05-21    1.09
Name: Open, dtype: float64
MESA, 0.89,   1.09,              582.7,                31.76


TOL
Date
2024-05-13    127.769997
2024-05-14    127.010002
2024-05-15    131.559998
2024-05-16    133.910004
2024-05-17    131.240005
2024-05-20    130.860001
2024-05-21    130.830002
Name: Open, dtype: float64
TOL, 131.24,   130.83,              -8.1,                25.85


RGTPQ
Date
2024-05-13    0.07
2024-05-14    0.03
2024-05-15    0.07
2024-05-16    0.03
2024-05-17    0.03
2024-05-20    0.03
2024-05-21    0.10
Name: Open, dtype: float64
RGTPQ, 0.03,   0.1,              6050.39,                86.43


XBIT
Date
2024-05-13    8.31
2024-05-14    8.66
2024-05-15    8.93
2024-05-16    8.60
2024-05-17    8.53
2024-05-20    8.50
2024-05-21    8.54
Name: Open, dtype: float64
XBIT, 8.53,   8.54,              3.04,                25.96


ONFO
Date
2024-05-13 

KeyboardInterrupt: 

In [102]:
invested_initial_value = 1045.17 
initial_value_per_ticker = invested_initial_value/len(intersection_companies)
total_initial_value = 0
total_final_value = 0
total_percentage = 0
if datetime.date(year, month, day).weekday() == 4:
    end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 4))
else:
    end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 2))
for tic in intersection_companies:
    yf_ticker = yf.download(tickers=tic, start=start_date, end =end_date, progress=False, ignore_tz=True)
    print('\n')
    print(tic)
    print(yf_ticker['Open'])
    open_values = yf_ticker['Open'].values
    initial_value = open_values[-3]
    final_value = open_values[-1]
    
    ## take some value when market is closed
    #yf_ticker = yf.download(tickers=tic, period = '1d', interval = '1m', progress=False, ignore_tz=True,  prepost= True)
    #final_value= yf_ticker['Open'].values[-1]
    
    percentage_difference = (final_value - initial_value)/initial_value*100
    print(f'{tic}, {np.round(initial_value, 2)},   {np.round(final_value, 2)},  \
            {np.round(initial_value_per_ticker*percentage_difference,2)},    \
            {np.round(initial_value_per_ticker + (initial_value_per_ticker*percentage_difference/100),2)}')
    total_initial_value += initial_value_per_ticker
    total_final_value += initial_value_per_ticker+(initial_value_per_ticker*percentage_difference/100)

# compare with spy
yf_ticker = yf.download(tickers='SPY', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_spy = (open_values[-1] - open_values[-3])/open_values[-3]*100

# compare with nasdaq
yf_ticker = yf.download(tickers='NDAQ', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_ndaq = (open_values[-1] - open_values[-3])/open_values[-3]*100


print('\n', total_initial_value, np.round(total_final_value,2), \
      np.round((total_final_value - total_initial_value)/total_initial_value*100,2), \
      np.round(percentage_difference_spy,2), np.round(percentage_difference_ndaq,2))

print(end_date)



STNE
Date
2024-03-08    16.650000
2024-03-11    16.730000
2024-03-12    16.959999
2024-03-13    17.180000
2024-03-14    17.250000
Name: Open, dtype: float64
STNE, 16.96,   17.25,              1787.15,                1063.04

 1045.17 1063.04 1.71 0.69 -0.83
2024-03-20


In [25]:
## just tradingview

year_i, month_i, day_i = 2024, 3, 4
year_f, month_f, day_f = 2024, 4, 12
up_to_ranking = 1

initial_date = datetime.date(year_i, month_i, day_i)
final_date = datetime.date(year_f, month_f, day_f)
current_date = initial_date
stream_1 = [initial_date]
stream_2 = []
index = 0
while (current_date < final_date):
    if current_date.weekday() == 4:
        next_date = current_date + datetime.timedelta(days = 3)
    else:
        if current_date.day == 28 and current_date.month == 3:
            next_date = current_date + datetime.timedelta(days = 4)
        else:
            next_date = current_date + datetime.timedelta(days = 1)
    
    if index % 2:
        stream_1.append(next_date)
    else:
        stream_2.append(next_date)

    current_date = next_date

    index += 1
print(stream_1, stream_2, '\n')




total_initial_sum = 1000
total_final_sum = 0

for stream in (stream_1, stream_2):


    invested_initial_value = total_initial_sum/2

    for date in stream:
        year = date.year
        month = date.month
        day = date.day 


        ticker_rank_tradingview = filter_by_lastrank('earnings', year, month, day, 'tradingview.com')

        if ticker_rank_tradingview is None: 
            continue
        ## tradingview
        total_number_of_companies = 0
        tradingview_companies = []
        for i in range(1,up_to_ranking + 1):
            ticker_rank_on212 = [x for x in ticker_rank_tradingview[f'{i}'] if x in trading212_tickers]
            tradingview_companies.extend(ticker_rank_on212)
            total_number_of_companies += len(ticker_rank_on212)


        if len(tradingview_companies):
            initial_value_per_ticker = invested_initial_value/len(tradingview_companies)
            total_initial_value = 0       
        else:
            initial_value_per_ticker = 0
            total_initial_value = 1
        
        total_final_value = 0
        total_percentage = 0      
        
        if datetime.date(year, month, day).weekday() == 0:
            if month == 4 and day == 1:
                start_date = str(datetime.date(year, month, day) - datetime.timedelta(days = 5))
            else:
                start_date = str(datetime.date(year, month, day) - datetime.timedelta(days = 3))
        else:
            start_date = str(datetime.date(year, month, day) - datetime.timedelta(days = 1))
        if datetime.date(year, month, day).weekday() == 4:
            
            end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 4))
        else:
            if day == 28 and month == 3:
                end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 5))
            elif day == 27 and month == 3:
                end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 2))

            else:
                end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 2))
        print(start_date, date, end_date)
        for tic in tradingview_companies:
            yf_ticker = yf.download(tickers=tic, start=start_date, end=end_date, progress=False, ignore_tz=True)
            open_values = yf_ticker['Open'].values
            print(yf_ticker['Open'])
            initial_value = open_values[-3]
            final_value = open_values[-1]
            
            ## take some value when market is closed
            #yf_ticker = yf.download(tickers=tic, period = '1d', interval = '1m', progress=False, ignore_tz=True,  prepost= True)
            #final_value= yf_ticker['Open'].values[-1]
            
            percentage_difference = (final_value - initial_value)/initial_value*100
            total_initial_value += initial_value_per_ticker
            total_final_value += initial_value_per_ticker+(initial_value_per_ticker*percentage_difference/100)

        # compare with spy
        yf_ticker = yf.download(tickers='SPY', start=start_date, end =end_date, progress=False, ignore_tz=True)
        open_values = yf_ticker['Open'].values
        final_value = open_values[-1]
        percentage_difference_spy = (open_values[-1] - open_values[-3])/open_values[-3]*100

        # compare with nasdaq
        yf_ticker = yf.download(tickers='NDAQ', start=start_date, end =end_date, progress=False, ignore_tz=True)
        open_values = yf_ticker['Open'].values
        final_value = open_values[-1]
        percentage_difference_ndaq = (open_values[-1] - open_values[-3])/open_values[-3]*100

        print(date, round(total_initial_value,2), round(total_final_value,2), round((total_final_value - total_initial_value)/total_initial_value*100,2))
        print('\n')
        invested_initial_value = total_final_value

    total_final_sum += total_final_value
    print(round(total_initial_sum/2,2), round(total_final_value,2), round((total_final_value - total_initial_sum/2)/(total_initial_sum/2)*100,2))
    print('\n\n')
print(round(total_initial_sum,2), round(total_final_sum,2), round((total_final_sum - total_initial_sum)/total_initial_sum*100,2))


[datetime.date(2024, 3, 4), datetime.date(2024, 3, 6), datetime.date(2024, 3, 8), datetime.date(2024, 3, 12), datetime.date(2024, 3, 14), datetime.date(2024, 3, 18), datetime.date(2024, 3, 20), datetime.date(2024, 3, 22), datetime.date(2024, 3, 26), datetime.date(2024, 3, 28), datetime.date(2024, 4, 2), datetime.date(2024, 4, 4), datetime.date(2024, 4, 8), datetime.date(2024, 4, 10), datetime.date(2024, 4, 12)] [datetime.date(2024, 3, 5), datetime.date(2024, 3, 7), datetime.date(2024, 3, 11), datetime.date(2024, 3, 13), datetime.date(2024, 3, 15), datetime.date(2024, 3, 19), datetime.date(2024, 3, 21), datetime.date(2024, 3, 25), datetime.date(2024, 3, 27), datetime.date(2024, 4, 1), datetime.date(2024, 4, 3), datetime.date(2024, 4, 5), datetime.date(2024, 4, 9), datetime.date(2024, 4, 11)] 

2024-03-01 2024-03-04 2024-03-06
Date
2024-03-01    7.06
2024-03-04    7.80
2024-03-05    7.60
Name: Open, dtype: float64
Date
2024-03-01     9.13
2024-03-04     9.25
2024-03-05    10.50
Name: Ope

/var/folders/9g/bzb1c3hn6jvdx57km19ky1y00000gn/T/ipykernel_72140/2129451699.py:118: RuntimeWarning: invalid value encountered in double_scalars
  print(date, round(total_initial_value,2), round(total_final_value,2), round((total_final_value - total_initial_value)/total_initial_value*100,2))


2024-04-10 0.0 0.0 nan


2024-04-11 2024-04-12 2024-04-16
Date
2024-04-11    0.84
2024-04-12    0.81
2024-04-15    0.68
Name: Open, dtype: float64


/var/folders/9g/bzb1c3hn6jvdx57km19ky1y00000gn/T/ipykernel_72140/2129451699.py:118: RuntimeWarning: invalid value encountered in double_scalars
  print(date, round(total_initial_value,2), round(total_final_value,2), round((total_final_value - total_initial_value)/total_initial_value*100,2))


2024-04-12 0.0 0.0 nan


500.0 0.0 -100.0



2024-03-04 2024-03-05 2024-03-07
Date
2024-03-04    5.20
2024-03-05    5.25
2024-03-06    5.10
Name: Open, dtype: float64
Date
2024-03-04    9.20
2024-03-05    8.94
2024-03-06    9.09
Name: Open, dtype: float64
Date
2024-03-04    4.92
2024-03-05    4.32
2024-03-06    4.10
Name: Open, dtype: float64
Date
2024-03-04    1.56
2024-03-05    1.50
2024-03-06    1.54
Name: Open, dtype: float64
Date
2024-03-04    7.00
2024-03-05    6.87
2024-03-06    6.60
Name: Open, dtype: float64
Date
2024-03-04    12.45
2024-03-05    12.11
2024-03-06    12.12
Name: Open, dtype: float64
Date
2024-03-04    26.32
2024-03-05    24.73
2024-03-06    25.33
Name: Open, dtype: float64
Date
2024-03-04    2.05
2024-03-05    2.06
2024-03-06    2.35
Name: Open, dtype: float64
Date
2024-03-04    2.08
2024-03-05    2.08
2024-03-06    2.15
Name: Open, dtype: float64
Date
2024-03-04    3.80
2024-03-05    3.66
2024-03-06    4.00
Name: Open, dtype: float64
2024-03-05 500.0 495.03 -0

/var/folders/9g/bzb1c3hn6jvdx57km19ky1y00000gn/T/ipykernel_72140/2129451699.py:118: RuntimeWarning: invalid value encountered in double_scalars
  print(date, round(total_initial_value,2), round(total_final_value,2), round((total_final_value - total_initial_value)/total_initial_value*100,2))


In [12]:
datetime.date(year, month, day)

datetime.date(2024, 3, 8)

In [7]:
stream_1

[datetime.date(2024, 3, 4),
 datetime.date(2024, 3, 6),
 datetime.date(2024, 3, 8),
 datetime.date(2024, 3, 12),
 datetime.date(2024, 3, 14),
 datetime.date(2024, 3, 18),
 datetime.date(2024, 3, 20),
 datetime.date(2024, 3, 22),
 datetime.date(2024, 3, 26),
 datetime.date(2024, 3, 28),
 datetime.date(2024, 4, 1),
 datetime.date(2024, 4, 3),
 datetime.date(2024, 4, 5),
 datetime.date(2024, 4, 9),
 datetime.date(2024, 4, 11)]

In [11]:
datetime

<module 'datetime' from '/Users/vini/miniforge3/envs/fin/lib/python3.8/datetime.py'>

In [30]:
## intersection

year_i, month_i, day_i = 2024, 3, 4
year_f, month_f, day_f = 2024, 3, 22
up_to_ranking = 2

initial_date = datetime.date(year_i, month_i, day_i)
final_date = datetime.date(year_f, month_f, day_f)
current_date = initial_date
stream_1 = [initial_date]
stream_2 = []
index = 0
while (current_date < final_date):
    if current_date.weekday() == 4:
        next_date = current_date + datetime.timedelta(days = 3)
    else:
        next_date = current_date + datetime.timedelta(days = 1)
    
    if index % 2:
        stream_1.append(next_date)
    else:
        stream_2.append(next_date)

    current_date = next_date

    index += 1
print(stream_1, stream_2, '\n')




total_initial_sum = 1000
total_final_sum = 0

for stream in (stream_1, stream_2):


    invested_initial_value = total_initial_sum/2

    for date in stream:
        year = date.year
        month = date.month
        day = date.day 



        ticker_rank_tradingview = filter_by_lastrank('earnings', year, month, day, 'tradingview.com')
        ticker_rank_zacks = filter_by_lastrank('earnings', year, month, day, 'zacks.com')
        up_to_ranking = 2

        total_number_of_companies = 0
        zacks_companies = []
        for i in range(1,up_to_ranking + 1):
            ticker_rank_on212 = [x for x in ticker_rank_zacks[f'{i}'] if x in trading212_tickers]
            zacks_companies.extend(ticker_rank_on212)
            total_number_of_companies += len(ticker_rank_on212)

        total_number_of_companies = 0
        tradingview_companies = []
        for i in range(1,up_to_ranking + 1):
            ticker_rank_on212 = [x for x in ticker_rank_tradingview[f'{i}'] if x in trading212_tickers]
            tradingview_companies.extend(ticker_rank_on212)
            total_number_of_companies += len(ticker_rank_on212)

        intersection_companies = list(set(zacks_companies) & set(tradingview_companies))


        if len(intersection_companies):
            initial_value_per_ticker = invested_initial_value/len(intersection_companies)
        else:
            initial_value_per_ticker = 0
        total_initial_value = 0
        total_final_value = 0
        total_percentage = 0        
        if datetime.date(year, month, day).weekday() == 0:
            start_date = str(datetime.date(year, month, day) - datetime.timedelta(days = 3))
        else:
            start_date = str(datetime.date(year, month, day) - datetime.timedelta(days = 1))
        if datetime.date(year, month, day).weekday() == 4:
            end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 4))
        else:
            end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 2))
        print(start_date, date, end_date)
        for tic in intersection_companies:
            yf_ticker = yf.download(tickers=tic, start=start_date, end=end_date, progress=False, ignore_tz=True)
            #print(tic)
            #print(yf_ticker['Open'][-3:], '\n')
            open_values = yf_ticker['Open'].values
            initial_value = open_values[-3]
            final_value = open_values[-1]
            
            ## take some value when market is closed
            #yf_ticker = yf.download(tickers=tic, period = '1d', interval = '1m', progress=False, ignore_tz=True,  prepost= True)
            #final_value= yf_ticker['Open'].values[-1]
            
            percentage_difference = (final_value - initial_value)/initial_value*100
            total_initial_value += initial_value_per_ticker
            total_final_value += initial_value_per_ticker+(initial_value_per_ticker*percentage_difference/100)

        # compare with spy
        yf_ticker = yf.download(tickers='SPY', start=start_date, end =end_date, progress=False, ignore_tz=True)
        open_values = yf_ticker['Open'].values
        final_value = open_values[-1]
        percentage_difference_spy = (open_values[-1] - open_values[-3])/open_values[-3]*100

        # compare with nasdaq
        yf_ticker = yf.download(tickers='NDAQ', start=start_date, end =end_date, progress=False, ignore_tz=True)
        open_values = yf_ticker['Open'].values
        final_value = open_values[-1]
        percentage_difference_ndaq = (open_values[-1] - open_values[-3])/open_values[-3]*100

        if len(intersection_companies): print(date, round(total_initial_value,2), round(total_final_value,2), round((total_final_value - total_initial_value)/total_initial_value*100,2))
        print('\n')
        invested_initial_value = total_final_value

    total_final_sum += total_final_value
    print(round(total_initial_sum/2,2), round(total_final_value,2), round((total_final_value - total_initial_sum/2)/(total_initial_sum/2)*100,2))
    print('\n\n')
print(round(total_initial_sum,2), round(total_final_sum,2), round((total_final_sum - total_initial_sum)/total_initial_sum*100,2))


[datetime.date(2024, 3, 4), datetime.date(2024, 3, 6), datetime.date(2024, 3, 8), datetime.date(2024, 3, 12), datetime.date(2024, 3, 14), datetime.date(2024, 3, 18), datetime.date(2024, 3, 20), datetime.date(2024, 3, 22)] [datetime.date(2024, 3, 5), datetime.date(2024, 3, 7), datetime.date(2024, 3, 11), datetime.date(2024, 3, 13), datetime.date(2024, 3, 15), datetime.date(2024, 3, 19), datetime.date(2024, 3, 21)] 

2024-03-01 2024-03-04 2024-03-06
2024-03-04 500.0 487.88 -2.42


2024-03-05 2024-03-06 2024-03-08
2024-03-06 487.88 553.98 13.55


2024-03-07 2024-03-08 2024-03-12
2024-03-08 553.98 541.09 -2.33


2024-03-11 2024-03-12 2024-03-14
2024-03-12 541.09 587.06 8.5


2024-03-13 2024-03-14 2024-03-16
2024-03-14 587.06 547.07 -6.81


2024-03-15 2024-03-18 2024-03-20
2024-03-18 547.07 490.77 -10.29


2024-03-19 2024-03-20 2024-03-22
2024-03-20 490.77 539.4 9.91


2024-03-21 2024-03-22 2024-03-26


500.0 0 -100.0



2024-03-04 2024-03-05 2024-03-07
2024-03-05 500.0 522.58 4.52


2024-0

/var/folders/9g/bzb1c3hn6jvdx57km19ky1y00000gn/T/ipykernel_1092/4185573573.py:112: RuntimeWarning: invalid value encountered in double_scalars
  if len(intersection_companies): print(date, round(total_initial_value,2), round(total_final_value,2), round((total_final_value - total_initial_value)/total_initial_value*100,2))
